In [206]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
from termcolor import colored
import os


data_path = './data.csv'

In [207]:
#random generate date function for validate
def generate_data():
    first_num = np.random.randint(low=0,high=30)
    second_num = np.random.randint(low=0,high=30)
    add = np.random.randint(low=0, high=100) 
    add2 = np.random.randint(low=0, high=100)
    if add <33:
        expression = str(first_num) + '+' + str(second_num)
        result = first_num+second_num
    elif add in range(33,66):
        expression = str(first_num) + '-' + str(second_num)
        result = first_num-second_num
    else :
        expression = str(first_num) + '*' + str(second_num)
        result = first_num*second_num
    
    third_num = np.random.randint(low=0,high=30)
    if add2 <33:
        expression = expression + '+' + str(third_num)+ "="
        result = result + third_num
    elif add2 in range(33,66):
        expression = expression + '-' + str(third_num)+ "="
        result = result - third_num
    else :
        expression = expression+ '*' + str(third_num) + "="
        if add <33:
            result = first_num+(second_num*third_num)
        elif add in range(33,66):
            result = first_num-(second_num*third_num)
        else:
            result = result * third_num
    #else:
    #    expression = str(first_num) + '/' + str(second_num)
    #    res = weird_division(first_num, second_num)
    #    result = str(round(res,2))

 
    return expression, result

generate_data()

('25*9+2=', 227)

In [208]:
df = pd.read_csv(os.path.join(data_path))
# 看一下前幾筆資料是什麼樣子

len(df)
#df=df[:640000]
df.head()

,src,tgt
0,18-14-10=,-6
1,11-14-0=,-3
2,0+14+23=,37
3,25+0-3=,22
4,10*15*28=,4200


<h1> Preprocessing

In [209]:

df['tgt']=df['tgt'].astype(str)
df['tgt']

0          -6
1          -3
2          37
3          22
4        4200
         ... 
19995      54
19996     513
19997     -12
19998    -244
19999     234
Name: tgt, Length: 20000, dtype: object

<h1>建立字典

In [210]:
# 一個dict把字符轉化成id
char_to_id = {}
# 把id轉回字符
id_to_char = {}

# 有一些必須要用的special token先添加進來(一般用來做padding的token的id是0)
char_to_id['<pad>'] = 0
char_to_id['<eos>'] = 1
id_to_char[0] = '<pad>'
id_to_char[1] = '<eos>'

#char_to_id={'<pad>': 0, '<eos>': 1, '2': 2, '7': 3, '3': 4, '9': 5, '5': 6, '6': 7, '0': 8, '-': 9, '1': 10, '8': 11, '=': 12, '+': 13, '4': 14}
#id_to_char={0: '<pad>', 1: '<eos>', 2: '2', 3: '7', 4: '3', 5: '9', 6: '5', 7: '6', 8: '0', 9: '-', 10: '1', 11: '8', 12: '=', 13: '+', 14: '4'}

# 把所有資料集中出現的token都記錄到dict中

for char in set(df['src'].str.cat()):
    ch_id = len(char_to_id)
    char_to_id[char] = ch_id
    id_to_char[ch_id] = char

vocab_size = len(char_to_id)
print('字典大小: {}'.format(vocab_size))
print('char_to_index : ', char_to_id)
print('index_to_char : ', id_to_char)

字典大小: 16
char_to_index :  {'<pad>': 0, '<eos>': 1, '1': 2, '8': 3, '0': 4, '=': 5, '2': 6, '-': 7, '7': 8, '6': 9, '+': 10, '*': 11, '4': 12, '5': 13, '9': 14, '3': 15}
index_to_char :  {0: '<pad>', 1: '<eos>', 2: '1', 3: '8', 4: '0', 5: '=', 6: '2', 7: '-', 8: '7', 9: '6', 10: '+', 11: '*', 12: '4', 13: '5', 14: '9', 15: '3'}


In [211]:
# 把資料集的所有資料都變成id
df['src_char_id_list'] = df['src'].apply(lambda text: [char_to_id[char] for char in list(text)] )+df['tgt'].apply(lambda text: [char_to_id[char] for char in list(text)]+ [char_to_id['<eos>']] )
df['tgt_char_id_list'] = df['tgt'].apply(lambda text: [char_to_id[char] for char in list(text)] + [char_to_id['<eos>']])
df[['src','tgt','src_char_id_list','tgt_char_id_list']].head()

,src,tgt,src_char_id_list,tgt_char_id_list
0,18-14-10=,-6,"[2, 3, 7, 2, 12, 7, 2, 4, 5, 7, 9, 1]","[7, 9, 1]"
1,11-14-0=,-3,"[2, 2, 7, 2, 12, 7, 4, 5, 7, 15, 1]","[7, 15, 1]"
2,0+14+23=,37,"[4, 10, 2, 12, 10, 6, 15, 5, 15, 8, 1]","[15, 8, 1]"
3,25+0-3=,22,"[6, 13, 10, 4, 7, 15, 5, 6, 6, 1]","[6, 6, 1]"
4,10*15*28=,4200,"[2, 4, 11, 2, 13, 11, 6, 3, 5, 12, 6, 4, 4, 1]","[12, 6, 4, 4, 1]"


# 超參數

|超參數|意義|數值|
|-|-|-|
|`batch_size`|單一 batch 的資料數|64|
|`epochs`|總共要訓練幾個 epoch|10|
|`embed_dim`|文字的 embedding 維度|50|
|`hidden_dim`|LSTM 中每個時間的 hidden state 維度|50|
|`lr`|Learning Rate|0.001|
|`grad_clip`|為了避免 RNN 出現梯度爆炸問題，將梯度限制範圍|1|

In [212]:
batch_size =400
epochs = 1000
embed_dim = 256
hidden_dim = 256
lr = 0.0001
grad_clip = 1

# 資料分批
- 使用 `torch.utils.data.Dataset` 建立資料產生的工具 `dataset`
- 再使用 `torch.utils.data.DataLoader` 對資料集 `dataset` 隨機抽樣並作為一個 batch


In [213]:
# 這裏的dataset是文本生成的dataset，輸入和輸出的資料都是文章
# 舉個例子，現在的狀況是：
# input:  A B C D E F 
# output: B C D E F <eos>
# 而對於加減法的任務：
# input:x 1 + 2 + 3 = 6  
#       y + 2 + 3 = 6 <eos>  
#         x:   5 +6 +7=18
#           y: + 6 +7=18<eos>  
# output: / / / / / 6 <eos>
# /的部分都不用算loss，主要是預測=的後面，這裏的答案是6，所以output是6 <eos>
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __getitem__(self, index):
        # input:  1 + 2 + 3 = 6
        # output: / / / / / 6 <eos>
        #x= 1+2+3=6
        #y= ppppp6<eos>
        #self.sequences.iloc[index] : 1+ 2+3=6<eos>
        x = self.sequences.iloc[index][:-1]
        y = self.sequences.iloc[index][1:]#self.sequences.iloc[index][self.sequences[index].index(char_to_id['='])+1:]
        # 找到=的索引
        index = y.index(char_to_id['='])
        # 將=左邊的元素設置為0
        y[:index+1] = [0] * (index+1)

        return x, y

    def __len__(self):
        return len(self.sequences)

def collate_fn(batch):

    batch_x = [torch.tensor(data[0]) for data in batch] # list[torch.tensor]
    batch_y = [torch.tensor(data[1]) for data in batch] # list[torch.tensor]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    equalidxs=torch.LongTensor([(data == char_to_id['=']).nonzero(as_tuple=True)[0] for data in batch_x])
    # torch.tensor
    # [[1968, 1891, 3580, ... , 0, 0, 0],
    #  [1014, 2242, 2247, ... , 0, 0, 0],
    #  [3032,  522, 1485, ... , 0, 0, 0]]
    #                       padding↑
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True, # shape=(batch_size, seq_len)
                                                  padding_value=char_to_id['<pad>'])

    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True, # shape=(batch_size, seq_len)
                                                  padding_value=char_to_id['<pad>'])

    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens ,equalidxs

In [214]:
dataset = Dataset(df['src_char_id_list'])

In [215]:
dataset.__getitem__(2)


([4, 10, 2, 12, 10, 6, 15, 5, 15, 8], [0, 0, 0, 0, 0, 0, 0, 15, 8, 1])

In [216]:
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          collate_fn=collate_fn
                                          )

In [217]:
len(data_loader)

50

In [218]:
#for i,x in enumerate(data_loader,):
#    print(x)

# 模型設計

## 執行順序
1. 將句子中的所有字轉換成 embedding
2. 按照句子順序將 embedding 丟入 LSTM
3. LSTM 的輸出再丟給 LSTM，可以接上更多層
4. 最後的 LSTM 所有時間點的輸出丟進一層 Fully Connected
5. 輸出結果所有維度中的最大者即為下一個字

## 損失函數
因為是類別預測，所以使用 Cross Entropy

## 梯度更新
使用 Adam 演算法進行梯度更新

In [219]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()

        # Embedding層
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        # RNN層
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        #self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
        #                                hidden_size=hidden_dim,
        #                                batch_first=True)
        
        #self.rnn_layer3 = torch.nn.LSTM(input_size=hidden_dim,
        #                                hidden_size=hidden_dim,
        #                                batch_first=True)

        # output層
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        

    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)

    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)

        # 假設有個tensor : tensor([[1, 2, 3, 4],
        #                        [9, 0, 0, 0]])
        # 輸出就是：PackedSequence(data=tensor([1, 9, 2, 3, 4]),
        #                         batch_sizes=tensor([2, 1, 1, 1]),
        #                         sorted_indices=None, unsorted_indices=None)
        # torch.nn.utils.rnn.pack_padded_sequence 會把batch當中的句子從長到短排序，建立如上所示的資料結構
        # 就像上一個例子一樣，RNN會先吃第一個batch內的第一個batch_size，看到這個地方的batch_size爲2，所以此時RNN會吃兩個token，輸出一個2Xhidden_dim的向量組
        # 然後看第二個batch_size, 此時爲1，少了一個，說明其中一個序列到頭了，那就取上一個輸出向量的第一個，再生成一個1Xhidden_dim的向量
        # [
        # [1,2,3],                data = [1,4,6,2,5,3]   output  data = [1p,4p,6p,2p,5p,3p]             [1p,2p,3p]
        # [4,5,0], => pack_padded_sequence => batch_sizes = [3,2,1] => RNN => batch_sizes = [3,2,1] => pad_packed_sequence => [4p,5p,0]
        # [6,0,0]         _                                                     [6p,0,0]
        # ]
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)

        batch_x, _ = self.rnn_layer1(batch_x)
        #batch_x, _ = self.rnn_layer2(batch_x)
        #batch_x, _ = self.rnn_layer3(batch_x)

        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)

        batch_x = self.linear(batch_x)

        return batch_x

    def generator(self, start_chars, max_len=200):
        char_list = []
        for char in start_chars:
            char_list.append(char_to_id[char])
    

        next_char = None
        # Move initial character list to the device
        char_list_tensor = torch.LongTensor(char_list).unsqueeze(0).to(device)
        # 生成的長度沒達到max_len就一直生
        while len(char_list) < max_len:

            x = self.embedding(char_list_tensor)
            _, (ht, _) = self.rnn_layer1(x)
          #  _, (ht, _) = self.rnn_layer2(ht)
          #  _, (ht, _) = self.rnn_layer3(ht)
            y = self.linear(ht)
            
            next_char = np.argmax(y.detach().cpu().numpy())

            # 如果看到新的token是<eos>就說明生成結束了，就停下
            if next_char == char_to_id['<eos>']:
                break

            char_list.append(next_char)
            char_list_tensor = torch.LongTensor(char_list).unsqueeze(0).to(device)

        return [id_to_char[ch_id] for ch_id in char_list]

In [221]:
torch.manual_seed(3)


model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)
#model.load_state_dict(torch.load('./models/model_state_dict.pt'))
model=model.to(device)
print(model)

CharRNN(
  (embedding): Embedding(16, 256, padding_idx=0)
  (rnn_layer1): LSTM(256, 256, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=16, bias=True)
  )
)


In [222]:
criterion = torch.nn.CrossEntropyLoss(reduction='mean',ignore_index=char_to_id['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 訓練
1. 最外層的 `for` 迴圈控制 `epoch`
    1. 內層的 `for` 迴圈透過 `data_loader` 取得 batch
        1. 丟給 `model` 進行訓練
        2. 預測結果 `batch_pred_y` 跟真正的答案 `batch_y` 進行 Cross Entropy 得到誤差 `loss`
        3. 使用 `loss.backward` 自動計算梯度
        4. 使用 `torch.nn.utils.clip_grad_value_` 將梯度限制在 `-grad_clip` &lt; &lt; `grad_clip` 之間
        5. 使用 `optimizer.step()` 進行更新（back propagation）
2. 每 `1000` 個 batch 就輸出一次當前的 loss 觀察是否有收斂的趨勢

In [223]:
from tqdm import tqdm
model = model.to(device)
model.train()
i = 0
for epoch in range(1, epochs+1):
    process_bar = tqdm(data_loader, desc=f"Training epoch {epoch}")
    
    for batch_x, batch_y, batch_x_lens, batch_y_lens,eqidex in process_bar:
        #print(batch_x)
        # 標準DL訓練
        
        optimizer.zero_grad()

        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        [64,7,15]
        #print(batch_pred_y)
        batch_pred_y=batch_pred_y[:, : ,:]
        #print(batch_pred_y.shape)
        batch_pred_y =  batch_pred_y.reshape(batch_size*batch_pred_y.shape[1],vocab_size)

        #print(batch_pred_y.shape)
        #print(batch_pred_y.shape)
        #print(batch_y)
        batch_y=batch_y.reshape(-1).to(device)
        #print(batch_y)
        # 計算損失          
        
        
        loss = criterion(batch_pred_y, batch_y)
        loss.backward() 
        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
        optimizer.step()
        
        
        i+=1
        if i%100==0:
            process_bar.set_postfix(loss=loss.item())


    #每20epoch validate一次
    if(epoch%20==0):
        validation_process_bar = tqdm(range(50))
        correctCount=0
        for i in validation_process_bar:
            expression,result=generate_data()
            pred = model.generator(expression)
            
            genStr=""
            for x in pred: 
                genStr+= x 
        
            #print(expression,result,genStr) 
            if(genStr==expression+str(result)):
                print(expression,genStr)
                correctCount=correctCount+1
        print("acc="+str(correctCount/50.0)) 
    if(epoch%500==0):
        torch.save(model, './models/model_epoch'+str(epoch)+'.pt')
           
        

100%|██████████| 50/50 [00:00<00:00, 342.25it/s]


17+23*20= 17+23*20=477
20-15*8= 20-15*8=-100
3*6*0= 3*6*0=0
6-17*1= 6-17*1=-11
7+17-25= 7+17-25=-1
27+16+0= 27+16+0=43
acc=0.12


 32%|███▏      | 16/50 [00:00<00:00, 157.74it/s]

7+21-18= 7+21-18=10
7-22*22= 7-22*22=-477
9+6-28= 9+6-28=-13
27*2+8= 27*2+8=62
19+9*20= 19+9*20=199
4-14*0= 4-14*0=4
6-16+28= 6-16+28=18
11+4+19= 11+4+19=34
19-3-26= 19-3-26=-10


100%|██████████| 50/50 [00:00<00:00, 138.29it/s]


...

12*21-4= 12*21-4=248
25*7+21= 25*7+21=196
9+3-0= 9+3-0=12
22+14+28= 22+14+28=64
25+25-7= 25+25-7=43
13-4-11= 13-4-11=-2
1+16-10= 1+16-10=7
9-16-11= 9-16-11=-18
2+10+5= 2+10+5=17
2+13*6= 2+13*6=80
0*20*22= 0*20*22=0
20+24-10= 20+24-10=34
13-5+9= 13-5+9=17
24-4+4= 24-4+4=24
0+14*6= 0+14*6=84
13*9+17= 13*9+17=134
1*26-26= 1*26-26=0
1-22-24= 1-22-24=-45
12+0+16= 12+0+16=28
2*20+3= 2*20+3=43
26*21*22= 26*21*22=12012
19+16+28= 19+16+28=63
2+1-29= 2+1-29=-26
25-24-12= 25-24-12=-11
15*0+19= 15*0+19=19
24-18-25= 24-18-25=-19
1+26-14= 1+26-14=13
2+8-24= 2+8-24=-14
12-26*21= 12-26*21=-534
3-17*20= 3-17*20=-337
5*24*8= 5*24*8=960
1+27*1= 1+27*1=28
6-0*0= 6-0*0=6
10-28+29= 10-28+29=11
acc=0.68


100%|██████████| 50/50 [00:00<00:00, 352.09it/s]


23-10*25= 23-10*25=-227
24+0+0= 24+0+0=24
27-22-6= 27-22-6=-1
10+19-11= 10+19-11=18
2*23-5= 2*23-5=41
7+22*19= 7+22*19=425
21+10-21= 21+10-21=10
18-26+13= 18-26+13=5
24+15-14= 24+15-14=25
12*5-25= 12*5-25=35
2*11+12= 2*11+12=34
24-0+29= 24-0+29=53
28-0-11= 28-0-11=17
9*19+21= 9*19+21=192
2*13+19= 2*13+19=45
16*10-0= 16*10-0=160
27-2-25= 27-2-25=0
23+17+28= 23+17+28=68
2-26+5= 2-26+5=-19
11*20-0= 11*20-0=220
28+15-17= 28+15-17=26
6+21-20= 6+21-20=7
2*12+10= 2*12+10=34
20-20*26= 20-20*26=-500
23*3*0= 23*3*0=0
25*16-15= 25*16-15=385
5-1+5= 5-1+5=9
19+21-1= 19+21-1=39
16+13+20= 16+13+20=49
25-3+18= 25-3+18=40
26+0-15= 26+0-15=11
0*26+29= 0*26+29=29
acc=0.64


Training epoch 841:  48%|████▊     | 24/50 [00:00<00:00, 42.05it/s]


KeyboardInterrupt: 

In [226]:
genStr=model.generator('10+12*13=')
print(genStr)

['1', '0', '+', '1', '2', '*', '1', '3', '=', '1', '7', '2']


In [227]:
validation_process_bar = tqdm(range(100))
correctCount=0
for i in validation_process_bar:
    expression,result=generate_data()
    
    pred = model.generator(expression)
    
    genStr=""
    for x in pred: 
         genStr+= x 


    if(genStr==expression+str(result)):
        
        print(colored(genStr,'green'),colored(result,'green'))
        correctCount=correctCount+1
    else:
        print(genStr,result)
print("acc="+str(correctCount/100.0))       

 52%|█████▏    | 52/100 [00:00<00:00, 257.52it/s]

23*6*1=134 138
24-25*11=-251 -251
2+12*29=354 350
18*26-18=450 450
5-29*14=-403 -401
23+12-4=31 31
22*25-10=530 540
27-12*19=-195 -201
4-29+20=-1 -5
6-0-4=0 2
14*1+8=22 22
9*13-23=90 94
7+0-2=3 5
2*9*1=16 18
28+16*24=424 412
20*3+29=93 89
22-22*26=-544 -550
28-10*9=-64 -62
17-14-28=-25 -25
9-17*22=-365 -365
20-20-24=-24 -24
7-3-9=-5 -5
21+13+11=45 45
13+6-26=-7 -7
11*14*7=1088 1078
13-2-12=-1 -1
28+23*17=421 419
29-0-10=19 19
17+17*25=446 442
1*11+5=16 16
29+16-1=44 44
16+24*14=354 352
6+13-12=7 7
6-16-26=-36 -36
8-6-27=-25 -25
22+19+28=69 69
10+13-15=8 8
16-16+23=23 23
20*26-23=497 497
26-20+14=20 20
25-23*29=-646 -642
29-29-12=-12 -12
20+10+7=37 37
8-9+28=27 27
15-5+29=39 39
3+12*11=135 135
16+21-5=32 32
22+12-14=20 20
20*8*3=460 480
8*22+15=191 191
13-16*22=-343 -339
19-12*29=-327 -329


100%|██████████| 100/100 [00:00<00:00, 277.50it/s]

6-22*25=-544 -544
25*20-12=478 488
24+1-20=5 5
28*15+21=431 441
19-23-6=-10 -10
23-19*9=-150 -148
25*0+4=4 4
8+0-21=-13 -13
0+12-24=-12 -12
3-10-5=-12 -12
12+19*2=48 50
20-13-0=7 7
26-1-23=2 2
16+22-6=32 32
19*28*8=4198 4256
22-5-11=6 6
29*11+0=319 319
23-0*16=21 23
17+27+10=54 54
8*29+15=247 247
26+5*19=127 121
7*23*8=1344 1288
20+10-22=8 8
18-0+26=44 44
21*18-27=349 351
3+26+15=44 44
2-5*7=-33 -33
0*4*27=0 0
8+10-24=-6 -6
8-11+24=21 21
16+1*0=16 16
5+0*23=5 5
0+10-22=-12 -12
23-28*14=-369 -369
9+13-20=2 2
8*21+26=192 194
14-29-23=-38 -38
28-25*3=-51 -47
21-9-13=-1 -1
28*28+5=777 789
27-21-0=8 6
18*28+21=517 525
21*21*8=3528 3528
1+10*22=221 221
17-22+29=24 24
3*29+1=84 88
28-11+27=44 44
27*23*27=16477 16767
acc=0.62


In [ ]:
torch.save(model.state_dict(), './models/model_state_dict.pt')